# LOG6308 : Tp2 - Approche contenu

- Clément Bernard (2096223)
- Ghaith Dekhili (1858454)

## Importations 

In [110]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import os
from sklearn.metrics.pairwise import cosine_similarity


from scipy.spatial import distance
import time
from sklearn import preprocessing
from numpy.linalg import multi_dot
from sklearn.model_selection import KFold

from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/clementbernard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/clementbernard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [61]:
DATA = os.path.join(os.getcwd(), 'data')
TABLE_PATH = os.path.join(DATA, 'citeseer.rtable')
ABSTRACT_PATH = os.path.join(DATA,'abstracts.csv')

## Data 


In [62]:
# Matrice d'adjacence
m = pd.read_table(TABLE_PATH, sep=' ')
# Number of articles
N = m.shape[0]

# Question 1

## Data

In [63]:
# The path where is the fold data
PATH_DATA = 'data'

# The items 
items = pd.read_csv(os.path.join(PATH_DATA, 'items.csv'), sep='|')
# User data 
u = pd.read_csv(os.path.join(PATH_DATA, 'u.csv'), sep='|')
# Votes of the user 
votes = pd.read_csv(os.path.join(PATH_DATA, 'votes.csv'), sep='|')

In [64]:
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
u

,id,age,gender,job,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [66]:
votes

,user.id,item.id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


## Creation of sparse matrix : User-item matrix

In [67]:
# The number of users 
N_USERS = u.shape[0] +1
# The number of items 
N_ITEMS = items.shape[0] +1 

In [68]:
def create_sparse_matrix(votes) : 
    ''' Create a User-Items sparse matrix '''
    # Create NaN for each items and users 
    data = {i : [np.nan for j in range(N_USERS )] for i in range(N_ITEMS)}
    def to_convert(x, data) :
        data[x['item.id']][x['user.id']] = x['rating']
        return None 
    votes.apply(to_convert , axis = 1 , args = [data])
    return pd.DataFrame(data)

In [69]:
user_item = create_sparse_matrix(votes)

In [70]:
user_item

,0,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# We drop the first user who has no grades
user_item= user_item.iloc[1:,1:]

In [72]:
def convert_to_binary(user_item):
    """
    returns a binary matrix by replacing votes equal or greater than 4 by 1 and 0 otherwise
    """
    user_item_binary = user_item.copy()
    # votes with 4 and 5 values become 1 and 0 otherwise 
    user_item_binary[user_item_binary < 4] = 0
    user_item_binary[user_item_binary >= 4] = 1
    return user_item_binary

In [73]:
def count_global_likes(user_item):
    """
    Counts global likes dislikes per item
    """
    #A df with number of likes and dislikes per item
    items_global_likes = pd.DataFrame(np.zeros((N_ITEMS-1, 2)), index = range(1,N_ITEMS))
    user_item_binary = convert_to_binary(user_item)
    for item in range(1,N_ITEMS):
        #Count number of 1(likes) and 0(dislikes) per item
        likes = user_item_binary[item].value_counts(normalize=False)
        #Fill with global number of likes and likes per item
        items_global_likes.loc[item,:] = likes
        #items_global_likes.loc[item,["dislikes","likes"]] = likes
    return user_item_binary, items_global_likes

In [74]:
def labelize_age(u):
    """
    labelize age based on users who are older than 25 are considered old
    rerturn a pandas Series with labelized ages
    """
    age_categorical = u[' age '].copy()
    age_categorical[u[' age '] > 25 ] = 'old'
    age_categorical[u[' age '] <= 25] = 'young'
    age_categorical = pd.DataFrame({'id ':[int(idx) for idx in u['id '].values],
                                    ' age ': age_categorical.values}, index = range(1,N_USERS))          
    return age_categorical

In [75]:
def return_dict_feature_ids(u,column, age = False):
    """
    returns a dictionary of a feature categories with their associated users ids
    """
    feature_per_ids = {}
    #A df with a feature and its associated ids users, for we you labelize ages 
    if age:
        df_feature = labelize_age(u)
    else:
        df_feature = u[['id ', column]].copy() 
    #group by feature per user_ids
    feature_per_ids = {name: group['id '].values for name,group in df_feature.groupby([column])}
    return feature_per_ids

In [76]:
def return_likes_dislikes_per_feature(u, user_item_binary, feature_per_ids):
    """
    returns likes dislikes matrices per job
    """
    #initializations
    item_likes_per_job = pd.DataFrame(np.nan, index = feature_per_ids.keys(), columns = range(1,N_ITEMS))
    item_dislikes_per_job = pd.DataFrame(np.nan, index = feature_per_ids.keys(), columns = range(1,N_ITEMS))
    #Loop over features and associated users ids
    for job, ids in feature_per_ids.items():
        item_likes_per_job.loc[job] = user_item_binary.loc[ids][user_item_binary.loc[ids]==1].count()
        item_dislikes_per_job.loc[job] = user_item_binary.loc[ids][user_item_binary.loc[ids]==0].count()
    
    return item_likes_per_job, item_dislikes_per_job

In [77]:
user_item_binary, items_global_likes = count_global_likes(user_item)
odds_likes = (items_global_likes.iloc[:,1]/items_global_likes.iloc[:,0])
proba_likes = odds_likes/(1+odds_likes)

In [78]:
#Returns a dictionary of jobs categories with their associated users ids
jobs_per_ids = return_dict_feature_ids(u, ' job ') 
#Return items likes and dislikes matrices per job
item_likes_per_job, item_dislikes_per_job = return_likes_dislikes_per_feature(u, user_item_binary, jobs_per_ids)
#calculate odds related to jobs
users_items_likes = (item_likes_per_job+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,1].replace(np.nan,0)+2),np.ones((1,item_likes_per_job.shape[0])))).T))
users_items_dislikes = (item_dislikes_per_job+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,0].replace(np.nan,0)+2),np.ones((1,item_likes_per_job.shape[0])))).T))
odds_jobs = pd.DataFrame(users_items_likes)/pd.DataFrame(users_items_dislikes)

In [79]:
#Returns a dictionary of gender categories with their associated users ids
gender_per_ids = return_dict_feature_ids(u, ' gender ')
#Return items likes and dislikes matrices per gender
item_likes_per_gender, item_dislikes_per_gender = return_likes_dislikes_per_feature(u, user_item_binary, gender_per_ids)
#calculate odds related to gender
users_items_likes_gender = (item_likes_per_gender+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,1].replace(np.nan,0)),np.ones((1,item_likes_per_gender.shape[0])))).T)+2)
users_items_dislikes_gender = (item_dislikes_per_gender+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,0].replace(np.nan,0)),np.ones((1,item_dislikes_per_gender.shape[0])))).T)+2)
odds_gender = pd.DataFrame(users_items_likes_gender)/pd.DataFrame(users_items_dislikes_gender)

In [80]:
#Returns a dictionary of age categories with their associated users ids
age_per_ids = return_dict_feature_ids(u, ' age ', age = True)
#Return items likes and dislikes matrices per age
item_likes_per_age, item_dislikes_per_age = return_likes_dislikes_per_feature(u, user_item_binary, age_per_ids)
#calculate odds related to age
users_items_likes_age = (item_likes_per_age + 1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,1].replace(np.nan,0)),np.ones((1,item_likes_per_age.shape[0])))).T)+2)
users_items_dislikes_age = (item_dislikes_per_age + 1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,0].replace(np.nan,0)),np.ones((1,item_dislikes_per_age.shape[0])))).T)+2)
odds_age = pd.DataFrame(users_items_likes_age)/pd.DataFrame(users_items_dislikes_age)

## Recommandation  de 10 films sur la base de trois catégories

In [81]:
def return_recommendations(job, gender, age, odds_likes, odds_jobs, odds_gender, odds_age):
    """
    return first 10 items recommendations based on input job, gender,age
    """
    odds = odds_likes*(odds_jobs.T)[job]*(odds_gender.T)[gender]*((odds_age.T))[age]
    return  (odds/(1+odds)).sort_values(ascending=False)[0:10]

In [82]:
return_recommendations('engineer', 'F', 'young', odds_likes = odds_likes, odds_jobs = odds_jobs, odds_gender = odds_gender, odds_age = odds_age)

483    0.967840
480    0.950883
115    0.928483
83     0.927111
107    0.927056
478    0.913225
114    0.911541
302    0.911324
918    0.908075
487    0.904690
dtype: float64

## Calcul des votes et du RMSE moyen 

In [83]:
a_c = labelize_age(u)
s = pd.DataFrame(u[' gender '].values, index = range(1,N_USERS), columns = [' gender '])
j = pd.DataFrame(u[' job '].values, index = range(1,N_USERS), columns = [' job '])
new_u = pd.concat([a_c , s , j], axis = 1)

In [84]:
def calculate_votes(user_item = user_item, new_u = new_u, odds_jobs = odds_jobs, odds_gender = odds_gender, odds_age = odds_age):
    """
    calculates votes based on probabilities
    """
    #A df which contains votes
    all_votes_predictions = pd.DataFrame(np.zeros((user_item.shape[0],user_item.shape[1])), index = range(1,N_USERS) ,columns = range(1,N_ITEMS))
    #A new df with age, gender and job in the needed labelization 
    s = pd.DataFrame(u[' gender '].values, index = range(1,N_USERS), columns = [' gender '])
    j = pd.DataFrame(u[' job '].values, index = range(1,N_USERS), columns = [' job '])
    a_c = labelize_age(u)
    new_u = pd.concat([a_c , s , j], axis = 1)
    #Calculate likes and dislikes averages
    likes_avgs = user_item[user_item >= 4].mean()
    dislikes_avgs = user_item[user_item < 4].mean()
    #Loop over the new df to calculate votes
    for index, row in new_u.iterrows():
        #Calculate odd by multiplying odds matrices
        all_odds = (odds_jobs.T[row[' job ']])*(odds_gender.T[row[' gender ']])*(odds_age.T[row[' age ']])
        #Calculates probabilities based on odds
        all_probs = all_odds/(1+all_odds)
        #Calculte votes per user
        user_votes = (all_probs*likes_avgs + (1-all_probs)*dislikes_avgs)
        #Fill in the df votes
        all_votes_predictions.loc[index,:] = user_votes
    
    return all_votes_predictions

In [85]:
def compute_rmse(prediction , user_item = user_item ) : 
    ''' Return the RMSE between the prediction and the user-item matrix '''
    # Index where the user_item is not null
    indexes_not_nan = user_item.notnull()
    rmse = np.sqrt(((prediction[indexes_not_nan] - user_item[indexes_not_nan])**2).mean())
    return rmse

In [86]:
all_votes_predictions = calculate_votes(user_item, new_u, odds_jobs, odds_gender, odds_age)

In [87]:
compute_rmse(all_votes_predictions, user_item).mean()

1.021163785917258

# Question 2

In [88]:
m

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10151,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Nous appliquons l'algorithme PageRank. On utilise la version matricielle, et pour eviter de diviser par 0, on remplit la diagonale à 0


In [89]:
def page_rank(m , d = 0.85, epsilon = 1e-5, max_iter = 10000) : 
    ''' Compute the PageRank score '''
    # Number of articles 
    N = m.shape[0]
    # PageRank values
    r = np.ones((1,N))
    # Initialise the condition 
    new_r = r + 10
    # Iteration 
    iteration = 0
    # Fill the diagonal to be 1, to avoid the division by 0 
    m_bis = m.copy()
    np.fill_diagonal(m_bis.values,1)
    # Condition : if the mean of difference is greater than epsilon
    condition = abs(r - new_r).mean() > epsilon 
    
    while condition and iteration < max_iter     : 
        # Get the number of output links 
        s = m_bis.sum(axis=0).values.reshape(-1,1)
        # Get the new PageRank vector
        new_r = (1-d)/N +  d * m_bis @ (r.T / s)                
        # Convert into numpy 
        new_r = new_r.values.T
        # Update the condition 
        condition = abs(r - new_r).mean() > epsilon
        r = new_r
        iteration+=1
        
    return pd.DataFrame(r.reshape(-1,), index = m.index, columns=['PageRank'])

In [90]:
r = page_rank(m, d= 0.85, epsilon = 1e-5, max_iter = 1000)

In [91]:
r

,PageRank
100299,0.001470
100967,0.000447
10151,0.000674
101705,0.001167
101863,0.000289
...,...
97863,0.006034
98185,0.000279
99113,0.002592
9947,0.000675


- Recommendation : prendre les articles maximum de PageRank multiplié par la matrice d'adjacence 

In [92]:
def get_recommendation(index, r = r, m=m,N= 10 ): 
    ''' Return recommendations for the index article from PageRank score '''
    # Get the element wise product between the PageRank and adjacent matrix
    idx = m[m.index == index].values * r.values.reshape(1,-1)
    idx = (-idx[0]).argsort()[:N]
    recommend = m.index[idx]
    return list(recommend)

In [93]:
get_recommendation(422908)

[155792, 3170, 131548, 241538, 17094, 124, 466838, 147460, 149673, 64835]

- Variante : etendre le sous-ensemble S aux références de références

In [94]:
m_ = m.values @ m.values + m.values.T

- On met a 1 les référencements qui sont multiples ou qui ont un chemin supérieur à 1 

In [95]:
m_[m_ >1] = 1
np.fill_diagonal(m_, 0)

In [96]:
new_m = m.copy()
new_m[:] = m_
new_m

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
98185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- On applique PageRank a ce nouvel espace 

In [97]:
new_r = page_rank(new_m, d= 0.85, epsilon = 1e-5, max_iter = 1000)

In [98]:
new_r

,PageRank
100299,0.000192
100967,0.000669
10151,0.001092
101705,0.000239
101863,0.000627
...,...
97863,0.002048
98185,0.000705
99113,0.000480
9947,0.000819


- On recommande ensuite 

In [99]:
get_recommendation(422908,r = new_r, m = new_m)

[311874, 19422, 297641, 464834, 83263, 17507, 177173, 226359, 26170, 7099]

# Question 3

- On considère pour cette question qu'un utilisateur est représenté par son vecteur dans la matrice d'adjacence
- On calcule ensuite la distance cosinus pour chaque utilisateur 
- La première recommendation sera forcément l'utilisateur lui-même 

In [100]:
def get_recommendation_sim(m = m , N=10) :
    ''' Return the top N recommendation using the cosine similarity '''
    # Get the similarity for each user 
    df_sim = pd.DataFrame(cosine_similarity(m,m), index = m.index, columns = m.columns)
    # Get the indexes that maximises the cosine similarity
    idx = (-df_sim.values).argsort()
    # Keep the N max
    idx = idx[:,:N]
    # Convert the indexes into article indexes
    article_idx = np.zeros(idx.shape)
    for i,index in enumerate(idx) : 
        for j,index_ in enumerate(index) : 
            article_idx[i,j] = m.index[index_]
    return pd.DataFrame(article_idx, index = m.index), df_sim

In [101]:
df_sim_top_10, df_sim = get_recommendation_sim(N=10)

In [102]:
# Matrice des distances 
df_sim

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
100967,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.288675,0.0,0.0,0.0,0.000000,0.000000
10151,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
101705,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
101863,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.000000,1.0,0.0,0.0,0.000000,0.000000
98185,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.235702,0.0,...,0.25,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.0,0.000000,0.000000
99113,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.000000
9947,0.0,0.0,0.0,0.0,0.0,0.136083,0.0,0.0,0.000000,0.0,...,0.00,0.000000,0.577350,0.0,0.000000,0.0,0.0,0.0,1.000000,0.333333


In [103]:
# Top 10 pour chaque article 
df_sim_top_10

,0,1,2,3,4,5,6,7,8,9
100299,100299.0,93456.0,25937.0,66991.0,164141.0,27921.0,120053.0,5101.0,3950.0,395364.0
100967,100967.0,281158.0,20858.0,298007.0,272596.0,63210.0,22178.0,252435.0,282066.0,43984.0
10151,10151.0,45228.0,93250.0,189479.0,130506.0,394952.0,3950.0,395364.0,40057.0,396568.0
101705,101705.0,42779.0,114285.0,88318.0,403583.0,402497.0,40057.0,40014.0,40009.0,397185.0
101863,101863.0,77030.0,134503.0,223844.0,224357.0,66776.0,145363.0,258875.0,135666.0,125089.0
...,...,...,...,...,...,...,...,...,...,...
97863,97863.0,171813.0,289801.0,170200.0,295968.0,8069.0,164643.0,281159.0,284612.0,503616.0
98185,98185.0,124.0,136975.0,83730.0,395364.0,1347.0,17507.0,468667.0,66285.0,373307.0
99113,99113.0,76418.0,236799.0,248936.0,394952.0,3950.0,395364.0,396568.0,393734.0,397021.0
9947,9947.0,31170.0,97150.0,139321.0,77621.0,50568.0,16117.0,164935.0,193029.0,251381.0


- 10 meilleures recommendations de 422908

In [104]:
df_sim_top_10[df_sim_top_10.index == 422908]

,0,1,2,3,4,5,6,7,8,9
422908,422908.0,96767.0,70445.0,149673.0,466838.0,155792.0,53632.0,497542.0,3175.0,17507.0


# Question 4

In [105]:
abstract = pd.read_csv(ABSTRACT_PATH)

In [106]:
abstract

,Unnamed: 0,Id,Titre,Auteurs,Description
0,1,124,Hybrid Automata: An Algorithmic Approach to th...,Costas Courcoubetis; Pei-hsin Ho; Rajeev Alur;...,We introduce the framework of hybrid automata ...
1,2,496,Representing Action: Indeterminacy and Ramific...,Enrico Giunchiglia; G. Neelakantan Kartha; Vla...,We define and study a high-level language for ...
2,3,712,A Decision Procedure for a Temporal Belief Logic,Michael Fisher; Michael Wooldridge;,. This paper presents a temporal belief logic ...
3,4,1186,A Comparative Evaluation of Sequential Feature...,David W. Aha; Richard L. Bankert;,Several recent machine learning publications d...
4,5,1347,Extending Promela and Spin for Real Time,Costas Courcoubetis; Stavros Tripakis;,The efficient representation and manipulation ...
...,...,...,...,...,...
1085,1086,548670,Null,NaN,NaN
1086,1087,549091,Null,NaN,NaN
1087,1088,561317,Null,NaN,NaN
1088,1089,570764,Null,NaN,NaN


- On ne considère que les descriptions 

In [111]:
def get_term_document_matrix(abstract = abstract ) : 
    ''' Return the term document matrix with TF-IDF '''
    # Number of documents 
    n_document = abstract.shape[0]
    # Vectorizer : the tf-idf vectorizer that does the Tokenisation and TF-IDF transformation
    vectorizer = TfidfVectorizer(strip_accents = 'unicode')
    # Get the corpus to apply the TF-IDF
    corpus = abstract['Description'].fillna(' ').values
    # Do the Tokenisation and TF-IDF
    tf_idf = vectorizer.fit_transform(corpus)
    # Vocabulary
    vocab = list(vectorizer.vocabulary_.keys())
    # Size of the vocabulary
    n_vocab = tf_idf.shape[1]
    # Term-document matrix 
    term_document = pd.DataFrame(tf_idf.todense().T, index = vocab, columns = abstract['Id'].values)
 
    return term_document

In [112]:
term_document = get_term_document_matrix()

In [113]:
term_document

,124,496,712,1186,1347,1948,2104,2111,2490,3156,...,543817,545708,547939,548310,548620,548670,549091,561317,570764,573595
we,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
introduce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
framework,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
of,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
byproduct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lar90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lt88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
glz89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
def get_recommendation_term_document(m = term_document , N=10) :
    ''' Return the top N recommendation using the cosine similarity '''
    # Get the similarity for each user 
    df_sim = pd.DataFrame(cosine_similarity(m.T,m.T), index = m.columns, columns = m.columns)
    # Get the indexes that maximises the cosine similarity
    idx = (-df_sim.values).argsort()
    # Keep the N max
    idx = idx[:,:N]
    # Convert the indexes into article indexes
    article_idx = np.zeros(idx.shape)
    for i,index in enumerate(idx) : 
        for j,index_ in enumerate(index) : 
            article_idx[i,j] = m.columns[index_]
    return pd.DataFrame(article_idx, index = m.columns), df_sim

In [115]:
df_sim_term_document, df_sim_tfidf = get_recommendation_term_document(m =term_document)

In [116]:
df_sim_term_document

,0,1,2,3,4,5,6,7,8,9
124,124.0,17507.0,134177.0,392298.0,314502.0,131548.0,7099.0,8434.0,149673.0,6497.0
496,496.0,18140.0,404934.0,403583.0,3177.0,59117.0,108101.0,434053.0,175470.0,391828.0
712,712.0,67284.0,179072.0,37710.0,149820.0,66776.0,67682.0,70445.0,41101.0,468667.0
1186,1186.0,135793.0,145880.0,208724.0,3156.0,122844.0,87538.0,138988.0,296226.0,158402.0
1347,1347.0,155792.0,110303.0,120172.0,8434.0,426325.0,6497.0,42779.0,297323.0,97410.0
...,...,...,...,...,...,...,...,...,...,...
548670,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
549091,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
561317,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
570764,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0


- 10 recommendations de l'article 422908

In [117]:
df_sim_term_document[df_sim_term_document.index == 422908]

,0,1,2,3,4,5,6,7,8,9
422908,422908.0,426325.0,120172.0,373307.0,96767.0,53595.0,110303.0,147460.0,155792.0,6497.0


# Question 5

In [135]:
df_sim_term_document

,0,1,2,3,4,5,6,7,8,9
124,124.0,17507.0,134177.0,392298.0,314502.0,131548.0,7099.0,8434.0,149673.0,6497.0
496,496.0,18140.0,404934.0,403583.0,3177.0,59117.0,108101.0,434053.0,175470.0,391828.0
712,712.0,67284.0,179072.0,37710.0,149820.0,66776.0,67682.0,70445.0,41101.0,468667.0
1186,1186.0,135793.0,145880.0,208724.0,3156.0,122844.0,87538.0,138988.0,296226.0,158402.0
1347,1347.0,155792.0,110303.0,120172.0,8434.0,426325.0,6497.0,42779.0,297323.0,97410.0
...,...,...,...,...,...,...,...,...,...,...
548670,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
549091,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
561317,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
570764,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0


In [217]:
def convert_recommendation_matrix(recommendations, m=m) : 
    ''' Convert the Top N recommendations to be represented as 1 in a 1090x1090 matrix '''
    # A 1090x1090 matrix with 1 when there is a recommendation
    m_rec = pd.DataFrame(np.zeros((m.shape[0], m.shape[0])), columns = list(m.index), index = list(m.index))
    
    for (index, row) in recommendations.iterrows() : 
        for k in row : 
            m_rec.loc[index,k] = 1 
    return m_rec

In [218]:
convert_recommendation_matrix(df_sim_term_document)

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100967,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10151,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101705,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101863,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
98185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
99113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [219]:
m_base = pd.read_table(TABLE_PATH, sep=' ')


def return_df_citations(m_base = m_base):  
    """
    returns a 2 columns df with refree/referenced papers
    """
    m_base = pd.read_table(TABLE_PATH, sep=' ')
    citations = []
    
    for i, (index, row) in enumerate(m_base.iterrows()):
        indexes = row[row ==1].index
        for k in indexes:
            if index != int(k) : 
                citations.append([index,k])
            
    return pd.DataFrame(citations, columns = ['referee','referenced'])

In [220]:
df_citations = return_df_citations(m_base = m_base)

In [221]:
df_citations

,referee,referenced
0,100299,170577
1,100299,527057
2,100967,173238
3,100967,185918
4,100967,275211
...,...,...
2816,9993,245669
2817,9993,248598
2818,9993,311874
2819,9993,353044


In [330]:
def k_fold(df_citations = df_citations, m = m_base  , N = 10) : 
    ''' Return the adjacent matrices for training and testing set, for N folds '''
    training = []
    testing = []
    np.fill_diagonal(m.values, 0)
    
    kf = KFold(n_splits = N , random_state = None, shuffle = True)
    
    for train_index, test_index in kf.split(df_citations) :
        
        m_copy = m.copy()
        m_test = m.copy()
                
        m_test.iloc[:,:] = 0
        
        test_indexes_citations = df_citations.iloc[test_index]
        for j,k in zip(test_indexes_citations['referee'].values, test_indexes_citations['referenced'].values) :            
            
            m_copy.loc[j,k] = 0
            m_test.loc[j,k] = 1
            
            
        m_copy.columns = m.index
        m_test.columns = m.index
        
        training.append(m_copy)
        testing.append(m_test)
        
    return training, testing 

In [252]:
training_index, testing_index = k_fold()

# Fonctions de recommendations 

In [328]:
def fn_recommendations_q3(matrice_adjacence_entrainement, *args, **kwargs) : 
    ''' Get the Top-20 recommendations : Similarity with adjacence matrix '''
    recommendations, _ = get_recommendation_sim(m = matrice_adjacence_entrainement,N=20)
    # Convert into DataFrame
    rec = pd.DataFrame(recommendations, columns = np.arange(20), index = matrice_adjacence_entrainement.index)
    # Convert to be of the size 1090 x 1090 with 1 when there is a recommendation 
    m_rec = convert_recommendation_matrix(rec)
    return m_rec

In [240]:
def fn_recommandations(matrice_adjacence_entrainement, *args, **kwargs) : 
    ''' Get the Top-20 recommendations : PageRank '''
    # Compute PageRank
    r = page_rank(matrice_adjacence_entrainement, d= 0.85, epsilon = 1e-5, max_iter = 1000)
    # Get the recommendations for every article
    recommendations = [get_recommendation(i, r = r, m=matrice_adjacence_entrainement,N= 20 ) for i in matrice_adjacence_entrainement.index ]
    # Convert into DataFrame
    rec = pd.DataFrame(recommendations, columns = np.arange(20), index = matrice_adjacence_entrainement.index)
    # Convert to be of the size 1090 x 1090 with 1 when there is a recommendation 
    m_rec = convert_recommendation_matrix(rec)
    return m_rec

In [344]:
def fn_recommandations_1(matrice_adjacence_entrainement, *args, **kwargs) : 
    ''' Get the Top-20 recommendations : PageRank++ '''
    # Compute PageRank
    m_ = matrice_adjacence_entrainement.values @ matrice_adjacence_entrainement.values + matrice_adjacence_entrainement.values.T
    m = matrice_adjacence_entrainement.copy()
    m[:] = m_
    r = page_rank(m, d= 0.85, epsilon = 1e-5, max_iter = 1000)
    # Get the recommendations for every article
    recommendations = [get_recommendation(i, r = r, m=m,N= 20 ) for i in m.index ]
    # Convert into DataFrame 
    rec = pd.DataFrame(recommendations, columns = np.arange(20), index = m.index)
    # Convert to be of the size 1090 x 1090 with 1 when there is a recommendation 
    m_rec = convert_recommendation_matrix(rec)
    return m_rec

In [326]:
def fn_recommendations_tfidf(matrice_adjacence_entrainement, *args, **kwargs) : 
    ''' Do the recommendations with the tf-idf method '''
    term_document = args[0][0]['term_document']
    
    recommendations,_ = get_recommendation_term_document(m = term_document, N=20)
    
    rec = pd.DataFrame(recommendations, columns = np.arange(20), index = m.index)
    
    m_rec = convert_recommendation_matrix(rec)
    
    return m_rec
    

In [306]:
recommandations = fn_recommandations(training_index[0])

In [278]:
def fn_taux_rappel(matrice_adjacence_test, recommandations) : 
    ''' Return the recall ratio '''
    return ((matrice_adjacence_test+recommandations)==2).sum().sum()/matrice_adjacence_test.sum().sum()

In [279]:
fn_taux_rappel(testing_index[0], recommandations)

0.014134275618374558

In [331]:
def get_recall(fn_recommandations = fn_recommandations, *args, **kwargs ) : 
    ''' Get the recall averaged 5 times '''
    recall = []
    for i in range(5) : 
        training_adjacences , testing_adjacences = k_fold()
        recall_val = []
        for matrice_train, matrice_test in zip(training_adjacences, testing_adjacences) : 
            recommandations = fn_recommandations(matrice_train, args, kwargs)
            current_recall = fn_taux_rappel(matrice_test, recommandations)
            recall_val.append(current_recall)
        recall.append(np.mean(recall_val))
    return np.mean(recall)

# Résultats de Recall

## Recall pour similarité classique 

In [337]:
get_recall(fn_recommendations_q3)

0.22063353632558952

## Recall pour PageRank standard

In [338]:
get_recall(fn_recommandations)

0.018575545698318423

## Recall pour PageRank étendu

In [345]:
get_recall(fn_recommandations_1)

0.36915470014785856

## Recall pour TF-IDF

In [340]:
args = {'term_document' : get_term_document_matrix()}

In [341]:
get_recall(fn_recommendations_tfidf, args)

0.30485602586271704